### Load dataframe

In [1]:
import pandas as pd

df = pd.read_csv("/Users/aleksandr/Desktop/Meta_Test.csv")
df = df.dropna()

### Initial cleaning

In [2]:
from clean import preprocess_tick_data

df_clean, df_diagnostics, outlier_counter = preprocess_tick_data(df)
df = df_clean
df = df.drop(columns="VOLATILITY")

Starting preprocessing with 570771 rows
After filtering trading hours: 282810 rows
After cleaning outliers: 282301 rows
Final clean dataset: 278585 rows

Outlier counts by detection method:
  zscore: 64
  extreme_deviation: 69
  isolated_point: 390
  price_reversal: 93
  timestamp_group: 34
  price_velocity: 3703
  suspicious_cluster: 52
  wavelet_outlier: 24


### Volatility estimation

In [3]:
from volatility_v1 import estimate_advanced_volatility

df = estimate_advanced_volatility(df)

Estimating advanced tick-level volatility for 278585 ticks...
Processing 278585 ticks...
Completed volatility estimation for 278585 ticks
Completed advanced tick-level volatility estimation


In [5]:
df.drop(columns=['return', "SYMBOL", "emd_vol", "sv_vol", 'log_price', 'smooth_vol', 'raw_vol'], inplace= True)
df.rename(columns={'filtered_vol' : 'Volatility', 
                  'TIMESTAMP':'Timestamp',
                   'VALUE' : 'Value',
                   'VOLUME' : 'Volume'}, inplace=True)
df.head()

,Timestamp,Value,Volume,Volatility
0,2025-01-30 09:30:00.740000+00:00,694.24,13.0,9.214653e-17
1,2025-01-30 09:30:00.740000+00:00,694.17,15.0,2.056060e-03
2,2025-01-30 09:30:00.740000+00:00,694.17,15.0,3.217889e-03
3,2025-01-30 09:30:00.740000+00:00,694.11,8.0,2.402309e-03
4,2025-01-30 09:30:00.740000+00:00,694.10,249.0,3.357669e-03


### Encoder Only Transformer Feature engine

In [ ]:
import os
import pandas as pd
from datetime import datetime
from Feature_engineering.feature_model_v1 import process_market_data

# Set up paths and configuration
df = df[:10000]
model_dir = '/Users/aleksandr/code/scripts/CronusV1/Feature_engineering/saved_models'

# Process data with the model 
features_df, model = process_market_data(
    df=df,
    model_dir=model_dir,
    retrain=True,
    num_epochs=10,
    context_length=50,
    num_attention_heads=8,
    num_encoder_layers=4,
    causal=True,
    temperature=0.5,
    grad_clip_norm=1.0
)

print(f"\nProcess completed successfully!")

Model will be trained and saved to: /Users/aleksandr/code/scripts/CronusV1/Feature_engineering/saved_models
Using causal mode. (Suitable for real-time applications)
Setting up feature extractor...
Extracting microstructure features...
Skip outlier clipping for Volatility: 596
Skip outlier clipping for price_change: 596
Skip outlier clipping for log_return: 596
Skip outlier clipping for time_delta: 596
Skip outlier clipping for trade_direction: 596
Skip outlier clipping for is_buy: 596
Skip outlier clipping for tick_imbalance: 596
Skip outlier clipping for jump_diffusion: 596
Skip outlier clipping for jump_magnitude: 596
Skip outlier clipping for jump_arrival: 596
Skip outlier clipping for kyle_lambda: 596
Skip outlier clipping for orderflow_imbalance: 596
Skip outlier clipping for momentum_short: 596
Skip outlier clipping for momentum_medium: 596
Skip outlier clipping for momentum_long: 596
Skip outlier clipping for price_range_short: 596
Skip outlier clipping for price_range_medium: 5

In [9]:
df = features_df
df.shape

(481, 20)